# Bibliotecas

In [ ]:
import numpy as np
import pandas as pd

# Fuzzification

In [ ]:
# Função que fuzzifica os valores crisp
def fuzzifica(data):
    # Create a fuzzified decision matrix - fdm
    fdm = np.zeros(shape=(data.shape[0], data.shape[1] * 3))
    k = 0
    for i in range(data.shape[1]):
        for j in range(data.shape[0]):
            # Extremely Poor
            if data[j, i] <= 0.05:
                fdm[j, k] = 0.0
                fdm[j, k + 1] = 0.0
                fdm[j, k + 2] = 0.1
            # Very Poor
            elif (data[j, i] > 0.05) and (data[j, i] <= 0.15):
                fdm[j, k] = 0.0
                fdm[j, k + 1] = 0.1
                fdm[j, k + 2] = 0.2
            # Poor to Very Poor
            elif (data[j, i] > 0.15) and (data[j, i] <= 0.25):
                fdm[j, k] = 0.1
                fdm[j, k + 1] = 0.2
                fdm[j, k + 2] = 0.3
            # Poor
            elif (data[j, i] > 0.25) and (data[j, i] <= 0.35):
                fdm[j, k] = 0.2
                fdm[j, k + 1] = 0.3
                fdm[j, k + 2] = 0.4
            # Poor to Fair
            elif (data[j, i] > 0.35) and (data[j, i] <= 0.45):
                fdm[j, k] = 0.3
                fdm[j, k + 1] = 0.4
                fdm[j, k + 2] = 0.5
            # Fair
            elif (data[j, i] > 0.45) and (data[j, i] <= 0.55):
                fdm[j, k] = 0.4
                fdm[j, k + 1] = 0.5
                fdm[j, k + 2] = 0.6
            # Fair to Good
            elif (data[j, i] > 0.55) and (data[j, i] <= 0.65):
                fdm[j, k] = 0.5
                fdm[j, k + 1] = 0.6
                fdm[j, k + 2] = 0.7
            # Good
            elif (data[j, i] > 0.65) and (data[j, i] <= 0.75):
                fdm[j, k] = 0.6
                fdm[j, k + 1] = 0.7
                fdm[j, k + 2] = 0.8
            # Good to Very Good
            elif (data[j, i] > 0.75) and (data[j, i] <= 0.85):
                fdm[j, k] = 0.7
                fdm[j, k + 1] = 0.8
                fdm[j, k + 2] = 0.9
            # Very Good
            elif (data[j, i] > 0.85) and (data[j, i] <= 0.95):
                fdm[j, k] = 0.8
                fdm[j, k + 1] = 0.9
                fdm[j, k + 2] = 1.0
            # Extremely Good
            elif (data[j, i] > 0.95) and (data[j, i] <= 1.0):
                fdm[j, k] = 0.9
                fdm[j, k + 1] = 1.0
                fdm[j, k + 2] = 1.0
        k += 3
    return fdm

# Fuzzy-MCDM

In [ ]:
# Check if the criteria is cost or benefit
def cost_benefit(cb):
    new_cb = np.zeros(len(cb)*3)
    k = 0
    for i in range(0, len(new_cb), 3):
        if cb[k] == 'max': #benefit
            new_cb[i] = 1
            new_cb[i+1] = 1
            new_cb[i+2] = 1
        else: #cost
            new_cb[i] = 0
            new_cb[i+1] = 0
            new_cb[i+2] = 0
        k = k + 1

    return new_cb

In [ ]:
# This function returns the aggregated weights for the criteria
# Each line should represent the evaluations of each DM to each criteria
#
# Get the lower value for a_ij, average for b_ij and max for c_ij
def aggregate_weights_crit(criteria):
    aggregate_weights = np.zeros(criteria.shape[1])

    for i in range(0, criteria.shape[1], 3):
        aggregate_weights[i] = np.min(criteria[:, i])
        aggregate_weights[i + 1] = (1 / criteria.shape[0]) * (sum(criteria[:, i + 1]))
        aggregate_weights[i + 2] = np.max(criteria[:, i + 2])

    return aggregate_weights

Fuzzy Topsis

In [ ]:
# Fuzzy TOPSIS Method for Multi-Criteria Decision Making Problems.
# Source: https://www.sciencedirect.com/science/article/pii/S187705091631273X

# This code implements the Fuzzy-TOPSIS with
# linear transformation (maximum) as normalization method.
# Input: alternatives -> array (each line is one alternative)
# Input: criteria -> array (weights for each criteria. each line is one DM) //
# cos_benefit -> "max" (benefit) or "min" (cost)

# TOPSIS method
def fuzzy_topsis(alternatives, criteria, cb):
    data = alternatives
    weights = criteria

    nrow = data.shape[0]
    ncol = data.shape[1]

    # Aggregate the weights
    weights = aggregate_weights_crit(weights)

    # Check if criteria is cost or benefit
    new_cb = cost_benefit(cb)

    # 3. Normalized fuzzy decision matrix
    N = np.zeros(shape=(nrow, ncol))
    for i in range(0, ncol, 3):
        if new_cb[i] == 1:
            denom = max(data[:, i + 2])
            N[:, i] = data[:, i] / denom
            N[:, i + 1] = data[:, i + 1] / denom
            N[:, i + 2] = data[:, i + 2] / denom
        else:
            denom = min(data[:, i + 2])
            N[:, i] = denom / data[:, i + 2]
            N[:, i + 1] = denom / data[:, i + 1]
            N[:, i + 2] = denom / data[:, i]

    # 4. Weighted normalized fuzzy decision matrix
    V = np.zeros(shape=(nrow, ncol))
    for i in range(nrow):
        for j in range(ncol):
            V[i, j] = N[i, j] * weights[j]

    # 5. FPIS and FNIS
    FPIS = np.ones(ncol)
    FNIS = np.zeros(ncol)

    # 6. Compute the distance from FPIS and FNIS
    distFPIS = np.zeros(shape=(nrow, ncol))
    distFNIS = np.zeros(shape=(nrow, ncol))
    for j in range(0, ncol, 3):
        distFPIS[:, j] = np.sqrt(
            (1 / 3) * ((V[:, j] - FPIS[j]) ** 2 + (V[:, j + 1] - FPIS[j + 1]) ** 2 + (V[:, j + 2] - FPIS[j + 2]) ** 2))
        distFNIS[:, j] = np.sqrt(
            (1 / 3) * ((V[:, j] - FNIS[j]) ** 2 + (V[:, j + 1] - FNIS[j + 1]) ** 2 + (V[:, j + 2] - FNIS[j + 2]) ** 2))

    dplus = np.zeros(nrow)
    dminus = np.zeros(nrow)
    for i in range(nrow):
        dplus[i] = np.sum(distFPIS[i, :])
        dminus[i] = np.sum(distFNIS[i, :])

    # 7. Compute the CCi
    CCi = np.zeros(nrow)
    for i in range(nrow):
        CCi[i] = dminus[i] / (dminus[i] + dplus[i])
    print(CCi)

    # Print the based on CCi values
    temp = (-CCi).argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(CCi))

    ranking = np.zeros((len(CCi), 3))
    for i in range(len(ranks)):
        ranking[i, 0] = i + 1
        ranking[i, 1] = CCi[i]
        ranking[i, 2] = np.abs(ranks[i]) + 1

    return ranking

Fuzzy Vikor

In [ ]:
# Aggregating importance criteria
def fuzzy_vikor(alternatives, criteria, cb, v):
    data = alternatives
    weights = criteria

    nrow = data.shape[0]
    ncol = data.shape[1]

    # Aggregate the weights
    weights = aggregate_weights_crit(weights)

    # Check if criteria is cost or benefit
    new_cb = cost_benefit(cb)

    # Determine fbest and fworst for each criteria
    posI = np.zeros(ncol)
    negI = np.zeros(ncol)
    for i in range(ncol):
        if new_cb[i] == 1:  # benefit
            posI[i] = max(data[:, i])
            negI[i] = min(data[:, i])
        else:
            posI[i] = min(data[:, i])
            negI[i] = max(data[:, i])

    # Calculate the normalized diference
    d = np.zeros(shape=(nrow, ncol))
    for i in range(0, ncol, 3):
        if new_cb[i] == 1:
            denom = posI[i + 2] - negI[i]
            d[:, i + 0] = (posI[i + 0] - data[:, i + 2]) / denom
            d[:, i + 1] = (posI[i + 1] - data[:, i + 1]) / denom
            d[:, i + 2] = (posI[i + 2] - data[:, i + 0]) / denom
        else:
            denom = negI[i + 2] - posI[i]
            d[:, i + 0] = (data[:, i + 0] - posI[i + 2]) / denom
            d[:, i + 1] = (data[:, i + 1] - posI[i + 1]) / denom
            d[:, i + 2] = (data[:, i + 2] - posI[i + 0]) / denom

    # Normalized fuzzy decision matrix
    V = np.zeros(shape=(nrow, ncol))
    for i in range(nrow):
        for j in range(ncol):
            V[i, j] = d[i, j] * weights[j]

    # Compute S and R
    S = np.zeros(shape=(nrow, 3))
    R = np.zeros(shape=(nrow, 3))
    aux1 = np.zeros(shape=(nrow, int(ncol/3)))
    aux2 = np.zeros(shape=(nrow, int(ncol/3)))
    aux3 = np.zeros(shape=(nrow, int(ncol/3)))

    # Get the l,m,u fuzzy values
    # Get l
    cont = 0
    for i in range(0, ncol, 3):
        for j in range(nrow):
            aux1[j, cont] = V[j, i]
        cont += 1
    # Get m
    cont = 0
    for i in range(1, ncol, 3):
        for j in range(nrow):
            aux2[j, cont] = V[j, i]
        cont += 1
    # Get u
    cont = 0
    for i in range(2, ncol, 3):
        for j in range(nrow):
            aux3[j, cont] = V[j, i]
        cont += 1

    for i in range(nrow):
        S[i, 0] = sum(aux1[i, :])
        S[i, 1] = sum(aux2[i, :])
        S[i, 2] = sum(aux3[i, :])
        R[i, 0] = max(aux1[i, :])
        R[i, 1] = max(aux2[i, :])
        R[i, 2] = max(aux3[i, :])

    # Compute Q
    Q = np.zeros(shape=(nrow, 3))
    Q1 = np.zeros(shape=(nrow, 3))
    Q2 = np.zeros(shape=(nrow, 3))

    denomS = max(S[:, 2]) - min(S[:, 0])
    Q1[:, 0] = (S[:, 0] - min(S[:, 2])) / denomS
    Q1[:, 1] = (S[:, 1] - min(S[:, 1])) / denomS
    Q1[:, 2] = (S[:, 2] - min(S[:, 0])) / denomS

    denomR = max(R[:, 2]) - min(R[:, 0])
    Q2[:, 0] = (R[:, 0] - min(R[:, 2])) / denomR
    Q2[:, 1] = (R[:, 1] - min(R[:, 1])) / denomR
    Q2[:, 2] = (R[:, 2] - min(R[:, 0])) / denomR

    if v == 1:
        Q = Q1
    elif v == 0:
        Q = Q2
    else:
        Q = v * Q1 + (1-v) * Q2

    def_s = np.zeros(nrow)
    Def_R = np.zeros(nrow)
    Def_Q = np.zeros(nrow)

    def_s = (S[:, 0] + S[:, 1] * 2 + S[:, 2]) / 4
    Def_R = (R[:, 0] + R[:, 1] * 2 + R[:, 2]) / 4
    Def_Q = (Q[:, 0] + Q[:, 1] * 2 + Q[:, 2]) / 4

    # Print results based on S, R and Q
    temp = (Def_Q).argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(Def_Q))

    ranking = np.zeros(shape=(len(Def_Q), 5))
    for i in range(len(ranks)):
        ranking[i, 0] = i + 1
        ranking[i, 1] = def_s[i]
        ranking[i, 2] = Def_R[i]
        ranking[i, 3] = Def_Q[i]
        ranking[i, 4] = np.abs(ranks[i]) + 1

    return ranking